In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
train_data = pd.read_csv("./data/train.csv")
test_data = pd.read_csv("./data/test.csv")

y_train = train_data.pop("price")

n_train = len(train_data)
all_data = pd.concat((train_data, test_data), axis=0)


In [ ]:
all_data.head()

In [ ]:
ax = sns.histplot(y_train, kde=True)

In [ ]:
from scipy import stats

y_train_log = np.log1p(y_train)
sns.histplot(y_train_log, kde=True)
plt.show()
ax = stats.probplot(y_train_log, plot=plt)
plt.show()


In [ ]:
numeric_columns = all_data.select_dtypes(include=[np.number]).columns.drop("id")
n_cols = int(np.ceil(len(numeric_columns) / 3))

fig, ax = plt.subplots(nrows=3, ncols=n_cols, figsize=(10, 10), squeeze=False)
for i, col_name in enumerate(numeric_columns):
    row = i // n_cols
    col = i % n_cols
    sns.histplot(all_data[col_name], kde=False, ax=ax[row][col])
    ax[row][col].set_title(col_name)
plt.tight_layout()
plt.show()


In [ ]:
numeric_columns = all_data.select_dtypes(include=[np.number]).columns.drop("id")
n_cols = int(np.ceil(len(numeric_columns) / 2))

fig, ax = plt.subplots(nrows=2, ncols=n_cols, figsize=(8, 8), squeeze=False)
for i, col_name in enumerate(numeric_columns):
    row = i // n_cols
    col = i % n_cols
    sns.boxplot(
        y=train_data[col_name], x=(y_train_log > y_train_log.median()).astype(int), ax=ax[row][col]
    )
    ax[row][col].set_title(col_name)
plt.tight_layout()
plt.show()


In [ ]:
categorical_columns = all_data.select_dtypes(include="object").columns
fig, axes = plt.subplots(ncols=3, figsize=(14, 4), squeeze=True)


orders = {
    "cut": ["Ideal", "Premium", "Very Good", "Good", "Fair"],
    "color": ["D", "E", "F", "G", "H", "I", "J"],
    "clarity": ["FL", "IF", "VVS1", "VVS2", "VS1", "VS2", "SI1", "SI2", "I1", "I2", "I3"],
}


for col_name, ax in zip(categorical_columns, axes):
    sns.boxplot(y=y_train, x=train_data[col_name], ax=ax, order=orders[col_name])
    ax.set_title(col_name)
    ax.set_axis_on()

plt.tight_layout()
plt.show()


It's interesting that worse cuts, colors and clarities present a tendency of higher prices. This is completely unintuitive. Likely, this is explained by the interaction with other variables. For instance, good clarity gemstones may be smaller on average, driving the price down.

In [ ]:
ax = sns.jointplot(x=train_data["carat"], y=y_train_log, kind="hex")
plt.show()
ax = sns.jointplot(x=train_data["carat"], y=y_train_log, hue=train_data["clarity"], hue_order=orders["clarity"], alpha=0.5)


In [ ]:
all_data["volume"] = all_data["x"] * all_data["y"] * all_data["z"]
all_data["volume"].round(2)

ax = sns.jointplot(x=all_data.iloc[:n_train]["volume"], y=y_train_log, hue=all_data.iloc[:n_train]["clarity"], hue_order=orders["clarity"], alpha=0.5)

For the same volume or carat, we can see that there's a tendency that gemstones with worse clarities have lower prices.

Let's encode ordinal variables and add new variables based on the existing ones. Check XXX for more details about the new variables.

In [ ]:
# https://www.kaggle.com/competitions/playground-series-s3e8/discussion/389207

orders_dict = {
    k: {vv: i for i, vv in enumerate(v)}
    for k, v in orders.items()
}

all_data['cut'] = all_data['cut'].apply(lambda x: orders_dict["cut"][x])
all_data['color'] = all_data['color'].apply(lambda x:orders_dict["color"][x])
all_data['clarity'] = all_data['clarity'].apply(lambda x:orders_dict["clarity"][x])
all_data["volume"] = all_data["x"] * all_data["y"] * all_data["z"]
all_data["surface_area"] = 2 * (all_data["x"] * all_data["y"] + all_data["y"] * all_data["z"] + all_data["z"] * all_data["x"])
# all_data["aspect_ratio_xy"] = all_data["x"] / all_data["y"]
# all_data["aspect_ratio_yz"] = all_data["y"] / all_data["z"]
# all_data["aspect_ratio_zx"] = all_data["z"] / all_data["x"]
all_data["diagonal_distance"] = np.sqrt(all_data["x"] ** 2 + all_data["y"] ** 2 + all_data["z"] ** 2)
all_data["relative_height"] = (all_data["z"] - all_data["z"].min()) / (all_data["z"].max() - all_data["z"].min())
all_data["relative_position"] = (all_data["x"] + all_data["y"] + all_data["z"]) / (all_data["x"] + all_data["y"] + all_data["z"]).sum()
all_data["volume_ratio"] = all_data["x"] * all_data["y"] * all_data["z"] / (all_data["x"].mean() * all_data["y"].mean() * all_data["z"].mean())
all_data["length_ratio"] = all_data["x"] / all_data["x"].mean()
all_data["width_ratio"] = all_data["y"] / all_data["y"].mean()
all_data["height_ratio"] = all_data["z"] / all_data["z"].mean()
all_data["sphericity"] = 1.4641 * (6 * all_data["volume"])**(2/3) / (1e-4 + all_data["surface_area"])
# all_data["compactness"] = all_data["volume"]**(1/3) / all_data["x"]

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


def train_cv_regressor(model_class, X, y, model_params, folds: int = 5, seed: int = 324):
    kf = KFold(n_splits=folds, random_state=seed, shuffle=True)
    models = []
    for i, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = model_class(**model_params, random_state=seed)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        val_pred = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_pred))
        print(f"Fold {i}: RMSE = {rmse:.3f}")
        models.append(model)
    return models


def train_regressor(model_class, X_train, X_val, y_train, y_val, model_params, seed: int = 324):
    model = model_class(**model_params, random_state=seed)
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    val_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    print(f"RMSE = {rmse:.3f}")
    return model


In [ ]:
import xgboost as xgb
import lightgbm as lgbm
import catboost as cb

xgb_params = {
    "n_estimators": 10_000,
    "learning_rate": 0.05,
    "max_depth": 10,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "early_stopping_rounds": 100,
    "objective": "reg:squarederror",
}

lgb_params = {
    'n_estimators': 2_000,
    'learning_rate': 0.05,
    'max_depth': 10,
    'subsample_for_bin': 20000,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'objective': 'regression',
}

cb_params = {
    'n_estimators': 2_000,
    'learning_rate': 0.05,
    'max_depth': 8,
    'loss_function': 'RMSE',
}

X = all_data.iloc[:n_train]
train_cv_regressor(xgb.XGBRegressor, X, y_train, xgb_params)
train_cv_regressor(lgbm.LGBMRegressor, X, y_train, lgb_params)
train_cv_regressor(cb.CatBoostRegressor, X, y_train, cb_params)

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone


class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds: int = 5, random_state: int = 342):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
        self.random_state = random_state

    # We again fit the data on clones of the original models
    def fit(self, X, y):
        X = X.values
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_state)

        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(
                    X[train_index],
                    y[train_index],
                    eval_set=[(X[holdout_index], y[holdout_index])],
                    verbose=False,
                )
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred

        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self

    # Do the predictions of all base models on the test data and use the averaged predictions as
    # meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack(
            [
                np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
                for base_models in self.base_models_
            ]
        )
        return self.meta_model_.predict(meta_features)

In [ ]:
# TODO rename y_train before
y = y_train

In [ ]:
from sklearn.linear_model import LinearRegression


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.8, random_state=342)

xgb_model = train_regressor(xgb.XGBRegressor, X_train, X_val, y_train, y_val, xgb_params)
cb_model = train_regressor(cb.CatBoostRegressor, X_train, X_val, y_train, y_val, cb_params)

meta_model = LinearRegression()
stacking_models = StackingAveragedModels([xgb_model, cb_model], meta_model=meta_model)
stacking_models.fit(X_train, y_train.values)
val_pred = stacking_models.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, val_pred))
print(f"Ensemble RMSE = {rmse:.3f}")